# [모듈 9.1] 추론 및 결과 분석

## 0. 기본 세이지 메이커 정보 및 기본 변수 로딩

In [1]:
import boto3
import sagemaker
import pandas as pd

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()

%store -r 
%store

Stored variables and their in-db values:
base_preproc_input_dir                 -> 'opt/ml/processing/input'
dataset_path                           -> 'opt/ml/processing/input/dataset.csv'
default_bucket                         -> 'sagemaker-ap-northeast-2-057716757052'
hyperparameters                        -> {'max_depth': '3', 'eta': '0.2', 'objective': 'bin
image_uri                              -> '366743142698.dkr.ecr.ap-northeast-2.amazonaws.com
input_data_uri                         -> 's3://sagemaker-ap-northeast-2-057716757052/fraud2
pipeline_endpoint_name                 -> 'all-pipeline-endpoint-0414'
preprocessing_code                     -> 'src/preprocessing.py'
preprocessing_code_dir                 -> 'src/preprocessing.py'
project_prefix                         -> 'fraud2scratch'
s3_dataset_path                        -> 's3://sagemaker-ap-northeast-2-057716757052/fraud2
s3_test_path                           -> 's3://sagemaker-ap-northeast-2-057716757052/fraud2
s3_tr

## 1. 환경 셋업

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# import os

# def split_X_y(test_preproc_path):
#     df = pd.read_csv(test_preproc_path)
#     y_test = df.iloc[:, 0].to_numpy()
#     df.drop(df.columns[0], axis=1, inplace=True)
    
#     return y_test, df

# s3_batch_test_path = f"{s3_dataset_path}/batch"
# print("s3_batch_test_path: ", s3_batch_test_path)


# # 전처리된 테스트 데이터 생성
# test_preproc_path = f"{test_preproc__dir_artifact}/test.csv"
# y_test, test_batch_df = split_X_y(test_preproc_path)

# # 로컬에 배치 변환 테스트 데이터 폴더 및 경로 생성 
# base_preproc_input_batch_dir = 'opt/ml/processing/input/batch'
# os.makedirs(base_preproc_input_batch_dir, exist_ok=True)
# batch_test_path = f"{base_preproc_input_batch_dir}/batch.csv"
# print("batch_test_path: ", batch_test_path)

# # 로컬에 배치 변환 파일 저장
# test_batch_df.to_csv(batch_test_path, index=None)

# # S3에 업로드
# input_batch_data_uri = sagemaker.s3.S3Uploader.upload(
#     local_path=batch_test_path, 
#     desired_s3_uri=s3_batch_test_path,
# )
# print("input_batch_data_uri: ", input_batch_data_uri)

## 2. 로컬에서 스크립트 실행

In [40]:
def get_payload_list(test_df, label_col):
    s = test_df.drop(columns=[label_col])
    s = s.to_csv(header=None, index=None)
    payload_list = s.splitlines()
    return payload_list

payload_list = get_payload_list(test_df, label_col='fraud')
y_test = test_df['fraud'].tolist()


In [36]:
import numpy as np
import boto3
sagemaker_boto_client = boto3.client('sagemaker')

from sagemaker.deserializers import CSVDeserializer
csv_deserializer = CSVDeserializer(accept='text/csv') # 디폴트가 accept가 'text/csv' 이지만 직관적 이유로 기술함.


from src.p_utils import is_available_endpoint, get_predictor, get_payload, predict


test_df = pd.read_csv(test_data_uri)    


In [37]:
%%time

if is_available_endpoint(pipeline_endpoint_name, verbose=False):
    pipeline_predictor = get_predictor(pipeline_endpoint_name, sagemaker_session, csv_deserializer)
    pred_prob_list = []
    payload_list = get_payload_list(test_df, label_col='fraud')
    for payload in payload_list:
        pred_prob = predict(pipeline_predictor, payload)
        pred_prob_list.append(float(pred_prob[0]))
    print("pred_prob_list: \n", pred_prob_list[0:5])
else:
    print("Wait for creating an endpoint")



pred_prob_list: 
 [0.010533533059060574, 0.010797908529639244, 0.03525015339255333, 0.0008525751763954759, 0.0029946493450552225]
CPU times: user 9.91 s, sys: 140 ms, total: 10.1 s
Wall time: 31.8 s


In [49]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

threshold = 0.4
predictions = [1 if e >= threshold else 0 for e in pred_prob_list ] 
    
#     print("y_test: ", y_test)
#     print("y_test length: ", len(y_test))    
#     print("predctions length: ", len(predictions))
#     print("predctions: ", predictions)    
#     print("predictions: ", predictions)
    # logging.info(f"{classification_report(y_true=y_test, y_pred = predictions)}")
print(f"{classification_report(y_true=y_test, y_pred = predictions)}")
cm = confusion_matrix(y_true= y_test, y_pred= predictions)    
print(cm)



              precision    recall  f1-score   support

           0       0.97      1.00      0.98      3876
           1       0.50      0.01      0.02       124

    accuracy                           0.97      4000
   macro avg       0.73      0.50      0.50      4000
weighted avg       0.95      0.97      0.95      4000

[[3875    1]
 [ 123    1]]
